In [1]:


%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import file_dic, plot_dic
from cmap import ncl_colormap
from mpl_toolkits.basemap import Basemap
%matplotlib qt



In [2]:
#set path for 
path_gen = '/media/onno/Algemeen/Thesis/'

In [3]:
#load RWP property files
file_rean_env = path_gen + file_dic['envelope']['ERA5']
file_rean_cp = path_gen + file_dic['phasespeed']['ERA5']
file_rean_t850 = path_gen + file_dic['T850']['ERA5']
file_GFS_env = path_gen + file_dic['envelope']['GFS']
file_GFS_cp = path_gen + file_dic['phasespeed']['GFS']
file_GFS_t850 = path_gen + file_dic['T850']['GFS']
file_ERA5RF_env = path_gen + file_dic['envelope']['ERA5RF']
file_ERA5RF_cp = path_gen + file_dic['phasespeed']['ERA5RF']
file_ERA5RF_t850 = path_gen + file_dic['T850']['ERA5RF']


rean_env_gen = xr.open_dataset(file_rean_env).sel(latitude=slice(90,0)).squeeze()
rean_cp_gen = xr.open_dataset(file_rean_cp).sel(latitude=slice(90,0)).squeeze()
rean_t850_gen = xr.open_dataset(file_rean_t850).sel(latitude=slice(90,0)).squeeze()

rean_env_gen =  rean_env_gen.assign_coords(longitude=(((rean_env_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_cp_gen =  rean_cp_gen.assign_coords(longitude=(((rean_cp_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_t850_gen = rean_t850_gen.assign_coords(longitude=(((rean_t850_gen.longitude + 180) % 360) - 180)).sortby('longitude')
rean_env_gen=rean_env_gen.rename({'longitude':'lon'});rean_env_gen=rean_env_gen.rename({'latitude':'lat'})
rean_cp_gen=rean_cp_gen.rename({'longitude':'lon'});rean_cp_gen=rean_cp_gen.rename({'latitude':'lat'})
rean_t850_gen=rean_t850_gen.rename({'longitude':'lon'});rean_t850_gen=rean_t850_gen.rename({'latitude':'lat'})
# rean_env_gen = rean_env_gen.sel(time=[bool(i) for i in rean_env_gen.time.dt.hour%24==0])
# rean_cp_gen = rean_cp_gen.sel(time=[bool(i) for i in rean_cp_gen.time.dt.hour%24==0])

ERA5RF_env_gen = xr.open_dataset(file_ERA5RF_env,decode_times=False).squeeze()
GFS_env_gen = xr.open_dataset(file_GFS_env,decode_times=False).squeeze()
ERA5RF_cp_gen = xr.open_dataset(file_ERA5RF_cp,decode_times=False).squeeze()
GFS_cp_gen = xr.open_dataset(file_GFS_cp,decode_times=False).squeeze()
ERA5RF_t850_gen = xr.open_dataset(file_ERA5RF_t850,decode_times=False).squeeze()
GFS_t850_gen = xr.open_dataset(file_GFS_t850,decode_times=False).squeeze()


GFS_init_time = pd.Timestamp('1800-01-01')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
GFS_env_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_env_gen.time.values]
GFS_cp_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_cp_gen.time.values]
GFS_t850_gen['time']=[pd.Timedelta(i,'hours')+GFS_init_time for i in GFS_t850_gen.time.values]
ERA5RF_env_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_env_gen.time.values]
ERA5RF_cp_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_cp_gen.time.values]
ERA5RF_t850_gen['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in ERA5RF_t850_gen.time.values]

GFS_env_gen = GFS_env_gen.rename({'lon':'longitude','lat':'latitude'}); GFS_cp_gen = GFS_cp_gen.rename({'lon':'longitude','lat':'latitude'})
ERA5RF_env_gen = ERA5RF_env_gen.rename({'lon':'longitude','lat':'latitude'}); ERA5RF_cp_gen = ERA5RF_cp_gen.rename({'lon':'longitude','lat':'latitude'})
ERA5RF_t850_gen = ERA5RF_t850_gen.rename({'lon':'longitude','lat':'latitude'});GFS_t850_gen = GFS_t850_gen.rename({'lon':'longitude','lat':'latitude'})

GFS_env_gen = GFS_env_gen.assign_coords(longitude=(((GFS_env_gen.longitude + 180) % 360) - 180)).sortby('longitude').sel(latitude=slice(90,0))
GFS_cp_gen = GFS_cp_gen.assign_coords(longitude=(((GFS_cp_gen.longitude + 180) % 360) - 180)).sortby('longitude').sel(latitude=slice(90,0))
GFS_t850_gen = GFS_t850_gen.assign_coords(longitude=(((GFS_t850_gen.longitude + 180) % 360) - 180)).sortby('longitude').sel(latitude=slice(90,0))
ERA5RF_env_gen = ERA5RF_env_gen.assign_coords(longitude=(((ERA5RF_env_gen.longitude + 180) % 360) - 180)).sortby('longitude').sel(latitude=slice(90,0))
ERA5RF_cp_gen = ERA5RF_cp_gen.assign_coords(longitude=(((ERA5RF_cp_gen.longitude + 180) % 360) - 180)).sortby('longitude').sel(latitude=slice(90,0))
ERA5RF_t850_gen = ERA5RF_t850_gen.assign_coords(longitude=(((ERA5RF_t850_gen.longitude + 180) % 360) - 180)).sortby('longitude').sel(latitude=slice(90,0))

# GFS_env_gen = GFS_env_gen.sel(lead=GFS_env_gen.lead.values[[bool(i) for i in GFS_env_gen.lead%24==0]])
# GFS_cp_gen = GFS_cp_gen.sel(lead=GFS_cp_gen.lead.values[[bool(i) for i in GFS_cp_gen.lead%24==0]])
# ERA5RF_env_gen = ERA5RF_env_gen.sel(lead=ERA5RF_env_gen.lead.values[[bool(i) for i in ERA5RF_env_gen.lead%24==0]])
# ERA5RF_cp_gen = ERA5RF_cp_gen.sel(lead=ERA5RF_cp_gen.lead.values[[bool(i) for i in ERA5RF_cp_gen.lead%24==0]])


GFS_env_gen = GFS_env_gen.rename({'latitude':'lat','longitude':'lon'})
GFS_cp_gen = GFS_cp_gen.rename({'latitude':'lat','longitude':'lon'})
GFS_t850_gen = GFS_t850_gen.rename({'latitude':'lat','longitude':'lon'})
ERA5RF_env_gen = ERA5RF_env_gen.rename({'latitude':'lat','longitude':'lon'})
ERA5RF_cp_gen = ERA5RF_cp_gen.rename({'latitude':'lat','longitude':'lon'})
ERA5RF_t850_gen = ERA5RF_t850_gen.rename({'latitude':'lat','longitude':'lon'})



In [4]:
coordinatez = [
(54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
# (66,58,6,14) #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
rankz = ['good','bad']
fcstz_env = [GFS_env_gen,ERA5RF_env_gen]
fcstz_cp = [GFS_cp_gen,ERA5RF_cp_gen]
modelz = ['GFS','ERA5RF']
seasonz = {'JJA':[6,7,8],
          'DJF':[12,1,2]}
lead_dayz = [3,5]
for lead_day in lead_dayz:
    for i,model in enumerate(modelz):
        fcst_env_gen = fcstz_env[i]
        fcst_cp_gen = fcstz_cp[i]
        for lat_0,lat_1,lon_0,lon_1 in coordinatez:
            list_df = []
            for season in seasonz:
                for event in eventz:
                    datez_event = np.load(path_gen+ 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
                    monthz = [i.month for i in pd.to_datetime(datez_event[:,0])]
                    datez_event = datez_event[np.isin(monthz,seasonz[season])]
    #                 for rank in rankz:


    #                     df = pd.read_csv(path+'GFS_T850/rank_forecast/seasonal/{}_forecasts{}_{}_lon_{}_{}_lat_{}_{}_{}_{}.txt'.format(rank,dataset_type,
    #                                                                                                                     event,
    #                                                                                                                   lon_0,lon_1,lat_0,lat_1,
    #                                                                                                                     season,model),
    #                                      index_col=0)
    #                     df.index = pd.to_datetime(df.index)

                    lon_0_RWP = lon_0 - 8
                    lon_1_RWP = lon_1 + 6
                    lat_0_RWP = lat_0 + 6
                    lat_1_RWP = lat_1 - 8
                    if lon_0_RWP>180:
                        lon_0_RWP = lon_0_RWP - 360;lon_1_RWP = lon_1_RWP - 360
                    all_meanz_env = []
                    all_meanz_cp = []
                    all_meanz_t850 = []
                    for date_event in datez_event:
                        begin_date_event,end_date_event = date_event
                        rean_env_sub = rean_env_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                    time=slice(begin_date_event,end_date_event+pd.Timedelta(18,'hours')))

                        rean_cp_sub = rean_cp_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                    time=slice(begin_date_event,end_date_event+pd.Timedelta(18,'hours')))
                        rean_t850_sub = rean_t850_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                    time=slice(begin_date_event,end_date_event+pd.Timedelta(18,'hours')))
                        all_meanz_env.append(rean_env_sub.mean(dim='time'))
                        all_meanz_cp.append(rean_cp_sub.mean(dim='time'))
                        all_meanz_t850.append(rean_t850_sub.mean(dim='time'))
                    rean_env_sub_all = xr.concat(all_meanz_env,dim='time')
                    rean_cp_sub_all = xr.concat(all_meanz_cp,dim='time')
                    rean_t850_sub_all = xr.concat(all_meanz_t850,dim='time')

                    rean_env_median = rean_env_sub_all.median(dim=['lat','lon']).v
                    rean_cp_median = rean_cp_sub_all.median(dim=['lat','lon']).v\
                    .where((rean_cp_sub_all.v.count(dim=['lat','lon'])/len(rean_cp_sub_all.lat)**2)>=.2)
                    rean_t850_median = rean_t850_sub_all.median(dim=['lat','lon']).t

                    rean_env_standardized = (rean_env_median - rean_env_median.mean())/rean_env_median.std()
                    rean_cp_standardized = (rean_cp_median - rean_cp_median.mean())/rean_cp_median.std()
                    rean_t850_standardized = (rean_t850_median - rean_t850_median.mean())/rean_t850_median.std()

                    df_norm = pd.DataFrame(index=datez_event[:,0],columns=['envelope','phasespeed','T850'])
                    df_norm.loc[:,'envelope'] = rean_env_standardized
                    df_norm.loc[:,'phasespeed'] = rean_cp_standardized
                    df_norm.loc[:,'T850'] = rean_t850_standardized
                    df_norm.loc['mean'] = df_norm.mean().values

                    df_av = pd.DataFrame(index=datez_event[:,0],columns=['envelope','phasespeed','T850'])
                    df_av.loc[:,'envelope'] = rean_env_median
                    df_av.loc[:,'phasespeed'] = rean_cp_median
                    df_av.loc[:,'T850'] = rean_t850_median
                    good_forecasts = pd.read_csv(path_gen+'/GFS_T850/rank_forecast/good_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                                      lon_0,lon_1,lat_0,lat_1,
                                                                                                                        season,model,lead_day),
                                         index_col=0).index
                    good_forecasts = pd.to_datetime(good_forecasts)
                    bad_forecasts = pd.read_csv(path_gen+'GFS_T850/rank_forecast/bad_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                                      lon_0,lon_1,lat_0,lat_1,
                                                                                                                        season,model,lead_day),
                                         index_col=0).index
                    bad_forecasts = pd.to_datetime(bad_forecasts)
                    df_norm_good = df_norm[df_norm.index.isin(good_forecasts)]
                    df_norm_bad = df_norm[df_norm.index.isin(bad_forecasts)]
                    df_av_good = df_av[df_av.index.isin(good_forecasts)]
                    df_av_bad = df_av[df_av.index.isin(bad_forecasts)]

                    df_av_good.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/good_forecasts_RWP_properties_AV_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
                    df_av_bad.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/bad_forecasts_RWP_properties_AV_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
                    df_norm_good.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/good_forecasts_RWP_properties_standardized_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
                    df_norm_bad.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/bad_forecasts_RWP_properties_standardized_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
                    #                 if len(all_meanz_cp)==0:
    #                     df.loc[rank]=[np.nan,np.nan]
    #                     df.loc[:,'envelope']=np.nan
    #                     df.loc[:,'phasespeed']=np.nan
    #                 else:
    #                     df.loc[rank] = [np.nan,np.nan,np.nan,np.nan]
    #                 list_df.append(df)
    #             with open(path+'fcst_RWP_properties/seasonal/forecasts_RWP_properties{}_lon_{}_{}_lat_{}_{}_RWP_total_{}.txt'.format(dataset_type,
    #                                                                                                                          lon_0,lon_1,lat_0,lat_1,model),'w') as foo:
    #                 for df in list_df:
    #                     foo.write(df.to_string())
    #                     foo.write('\n')


In [6]:


coordinatez = [
(54,46,6,14),#Germany
# (44,36,352,360), #Spain
# (54,46,26,34), #Ukraine
(58,50,352,360), #UK
# (42,34,28,36), #Turkey
# (68,60,22,30), #Finland
# (66,58,6,14) #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region
dataset_type = '_split_dataset'
eventz = ['persistent_hw','persistent_cw','short_hw','short_cw']
rankz = ['good','bad']
fcstz_env = [GFS_env_gen,ERA5RF_env_gen]
fcstz_cp = [GFS_cp_gen,ERA5RF_cp_gen]
fcstz_t850 = [GFS_t850_gen,ERA5RF_t850_gen]
modelz = ['GFS','ERA5RF']
seasonz = {'JJA':[6,7,8],
          'DJF':[12,1,2]}
lead_dayz = [3,5]

for i,model in enumerate(modelz):
    for lead_day in lead_dayz:
        fcst_env_gen = fcstz_env[i]
        fcst_cp_gen = fcstz_cp[i]
        fcst_t850_gen = fcstz_t850[i]
        for lat_0,lat_1,lon_0,lon_1 in coordinatez:
            list_df = []
            for season in seasonz:
                for event in eventz:
                    datez_event = np.load(path_gen+ 'GFS_T850/{}_lon_{}_{}_lat_{}_{}.npy'.format(event,lon_0,lon_1-2,lat_0-2,lat_1))
                    monthz = [i.month for i in pd.to_datetime(datez_event[:,0])]
                    datez_event = datez_event[np.isin(monthz,seasonz[season])]
    #                 for rank in rankz:


    #                     df = pd.read_csv(path+'GFS_T850/rank_forecast/seasonal/{}_forecasts{}_{}_lon_{}_{}_lat_{}_{}_{}_{}.txt'.format(rank,dataset_type,
    #                                                                                                                     event,
    #                                                                                                                   lon_0,lon_1,lat_0,lat_1,
    #                                                                                                                     season,model),
    #                                      index_col=0)
    #                     df.index = pd.to_datetime(df.index)

                    lon_0_RWP = lon_0 - 8
                    lon_1_RWP = lon_1 + 6
                    lat_0_RWP = lat_0 + 6
                    lat_1_RWP = lat_1 - 8
                    if lon_0_RWP>180:
                        lon_0_RWP = lon_0_RWP - 360;lon_1_RWP = lon_1_RWP - 360
                    all_meanz_env = []
                    all_meanz_cp = []
                    all_meanz_t850 = []
                    df_av = pd.DataFrame(index=datez_event[:,0],columns=['envelope','phasespeed','T850'])
                    for date_event in datez_event:
                        begin_date_event,end_date_event = date_event
                        begin_date_fcst = begin_date_event - pd.Timedelta(lead_day*24,'hours')
                        end_date_fcst = begin_date_fcst + pd.Timedelta(9*24,'hours')

                        if end_date_event>end_date_fcst:
                            end_date_event = end_date_fcst
                        end_lead_hours_object = ((end_date_event - begin_date_fcst) + pd.Timedelta(18,'h'))
                        end_lead_hours = end_lead_hours_object.days*24 + end_lead_hours_object.seconds//3600 
                        try:
                            rean_env_sub = rean_env_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                        time=slice(begin_date_event,end_date_event+pd.Timedelta(18,'h')))
                            if model == 'ERA5RF':
                                rean_env_sub = rean_env_sub.sel(time=pd.date_range(begin_date_event,end_date_event+pd.Timedelta(18,'h'),freq='12H'))
                            fcst_env_sub = fcst_env_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                        time=begin_date_fcst,lead=slice(lead_day*24,end_lead_hours))
                            diff_fcst_env = fcst_env_sub - rean_env_sub.v.values
                            diff_env_median = diff_fcst_env.median(dim=['lat','lon']).v
                            
                            rean_t850_sub = rean_t850_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                        time=slice(begin_date_event,end_date_event+pd.Timedelta(18,'h')))
                            fcst_t850_sub = fcst_t850_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                        time=begin_date_fcst,lead=slice(lead_day*24,end_lead_hours))
                            if model == 'ERA5RF':
                                rean_t850_sub = rean_t850_sub.sel(time=pd.date_range(begin_date_event,end_date_event+pd.Timedelta(18,'h'),freq='12H'))
                            diff_fcst_t850 = fcst_t850_sub - rean_t850_sub.t.values
                            diff_t850_median = diff_fcst_t850.median(dim=['lat','lon']).t
                            
                            if model == 'GFS':

                                rean_cp_sub = rean_cp_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                            time=slice(begin_date_event,end_date_event+pd.Timedelta(18,'h')))
                                fcst_cp_sub = fcst_cp_gen.sel(lat=slice(lat_0_RWP,lat_1_RWP),lon=slice(lon_0_RWP,lon_1_RWP),
                                                            time=begin_date_fcst,lead=slice(lead_day*24,end_lead_hours))
                                diff_fcst_cp = fcst_cp_sub - rean_cp_sub.v.values
                                diff_cp_median = diff_fcst_cp.median(dim=['lat','lon']).v\
                                .where((diff_fcst_cp.v.count(dim=['lat','lon'])/len(diff_fcst_cp.lat)**2)>=.2)
                                
                                df_av.loc[begin_date_event,'phasespeed'] = float(diff_cp_median.mean())

                            df_av.loc[begin_date_event,'envelope'] = float(diff_env_median.mean())
                            df_av.loc[begin_date_event,'T850'] = float(diff_t850_median.mean())

                        except KeyError:
                            df_av.loc[begin_date_event,'envelope'] = np.nan
                            df_av.loc[begin_date_event,'T850'] = np.nan
                            if model == 'GFS':
                                df_av.loc[begin_date_event,'phasespeed'] = np.nan
                    df_norm = (df_av - df_av.mean())/df_av.std()

    #                     all_meanz_env.append(diff_fcst_env.mean(dim='lead'))
    #                     all_meanz_cp.append(diff_fcst_cp.mean(dim='lead'))
    #                 diff_env_sub_all = xr.concat(all_meanz_env,dim='time')
    #                 diff_cp_sub_all = xr.concat(all_meanz_cp,dim='time')

    #                 diff_env_median = diff_env_sub_all.median(dim=['lat','lon']).v
    #                 diff_cp_median = diff_cp_sub_all.median(dim=['lat','lon']).v\
    #                 .where((diff_cp_sub_all.v.count(dim=['lat','lon'])/len(diff_cp_sub_all.lat)**2)>=.2)



    #                 df.loc[:,'envelope'] = diff_env_standardized
    #                 df.loc[:,'phasespeed'] = diff_cp_standardized
    #                 df.loc['mean'] = df.mean().values
                    good_forecasts = pd.read_csv(path_gen+'/GFS_T850/rank_forecast/good_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                                      lon_0,lon_1,lat_0,lat_1,
                                                                                                                        season,model,lead_day),
                                         index_col=0).index
                    good_forecasts = pd.to_datetime(good_forecasts)
                    bad_forecasts = pd.read_csv(path_gen+'GFS_T850/rank_forecast/bad_forecasts_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                                      lon_0,lon_1,lat_0,lat_1,
                                                                                                                        season,model,lead_day),
                                         index_col=0).index
                    bad_forecasts = pd.to_datetime(bad_forecasts)
                    df_av_good = df_av[df_av.index.isin(good_forecasts)]
                    df_av_bad = df_av[df_av.index.isin(bad_forecasts)]
                    df_norm_good = df_norm[df_av.index.isin(good_forecasts)]
                    df_norm_bad = df_norm[df_av.index.isin(bad_forecasts)]
                    df_av_good.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/good_forecasts_errors_AV_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
                    df_av_bad.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/bad_forecasts_errors_AV_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
                    df_norm_good.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/good_forecasts_errors_standardized_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
                    df_norm_bad.to_csv(path_gen+'fcst_RWP_properties/NW_Europe/bad_forecasts_errors_standardized_{}_lon_{}_{}_lat_{}_{}_{}_{}_lead_day_{}.txt'.format(event,
                                                                                                            lon_0,lon_1,lat_0,lat_1,
                                                                                                                 season,model,lead_day))
    #                 if len(all_meanz_cp)==0:
    #                     df.loc[rank]=[np.nan,np.nan]
    #                     df.loc[:,'envelope']=np.nan
    #                     df.loc[:,'phasespeed']=np.nan
    #                 else:
    #                     df.loc[rank] = [np.nan,np.nan,np.nan,np.nan]
    #                 list_df.append(df)
    #             with open(path+'fcst_RWP_properties/seasonal/forecasts_RWP_properties{}_lon_{}_{}_lat_{}_{}_RWP_total_{}.txt'.format(dataset_type,
    #                                                                                                                          lon_0,lon_1,lat_0,lat_1,model),'w') as foo:
    #                 for df in list_df:
    #                     foo.write(df.to_string())
    #                     foo.write('\n')

